- 개체명 인식 : NER
    - 텍스트에서 특정 의미를 가진 단어나 구절을 찾아내고 분류하는 작업

In [2]:
# 홍길동은 2025년 11월 19일 서울시청에서 삼성전자 직원을 만났다.
# 홍길동 - [인명]
# 2025년 11월 19일 - [일시]
# 서울시청 - [지명]
# 삼성전자 - [ 기관명]

# 활용분야
    #뉴스기사 : 기사에서 인물, 장소, 기관 자동추출
    # 의료문서 : 병명, 약물명, 증상
    # 계약서 : 회사명, 날자, 금액
    # 챗봇 : 사용자 질문에 핵심정보 파악
# BIO 태깅
# B(begin) 개체 시작
# I(inside) 개체 내부
# O(outside) 개체가 아님

In [3]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

In [4]:
# Bio 태깅
tokens = ["김철수는", "2024년", "1월", "15일", "서울시청에서", "삼성전자", "직원을", "만났다"]
bio_tags = ["B-PER", "B-DAT", "I-DAT", "I-DAT", "B-LOC", "B-ORG", "O", "O"]
for token, tag in zip(tokens, bio_tags):
  if tag.startswith('B-'):
    desc = f"'{tag[2:]}' 개체의 시작"
  elif tag.startswith('I-'):
    desc = f"'{tag[2:]}' 개체의 내부"
  else:
    desc = "개체가 아님"
  print(f"{token:12} | {tag:8} | {desc}")

김철수는         | B-PER    | 'PER' 개체의 시작
2024년        | B-DAT    | 'DAT' 개체의 시작
1월           | I-DAT    | 'DAT' 개체의 내부
15일          | I-DAT    | 'DAT' 개체의 내부
서울시청에서       | B-LOC    | 'LOC' 개체의 시작
삼성전자         | B-ORG    | 'ORG' 개체의 시작
직원을          | O        | 개체가 아님
만났다          | O        | 개체가 아님


In [5]:
# 학습데이터
train_sentences = [
    ["김철수는", "서울에", "산다"],
    ["이영희는", "2024년에", "부산으로", "이사했다"],
    ["삼성전자는", "대한민국의", "대기업이다"],
    ["박지성은", "축구선수다"],
    ["2025년", "1월", "1일은", "새해다"],
]

train_labels = [
    ["B-PER", "B-LOC", "O"],
    ["B-PER", "B-DAT", "B-LOC", "O"],
    ["B-ORG", "B-LOC", "O"],
    ["B-PER", "O"],
    ["B-DAT", "I-DAT", "I-DAT", "O"],
]

In [ ]:
# %pip install protobuf==3.20.3

Note: you may need to restart the kernel to use updated packages.


In [7]:
from ast import mod
# 토크나이져
MODEL_NAME = 'skt/kobert-base-v1'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
text = '김철수는 서울에 산다'
#토크나이져
tokens = tokenizer.tokenize(text)
# 인코딩
encoded = tokenizer(text,return_tensors='pt')
encoded

c:\Users\khh11\miniconda3\envs\pyt_env\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'input_ids': tensor([[517, 490, 494,   0, 517,   0, 491,   0, 491,   0, 517,   0,   0,   0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [8]:
# NER 모델 3단계로 구성
# 1. 입력 텍스트
# 2. koBERT 인코더
# 3. 분류기(Linear)
# 4. 출력 라벨 B-PER B-LOC B-ORG I-PER I-LOC I-ORG O

In [9]:
import numpy as np
results = []
for i in train_labels:
    results.append(i)
results = np.array(results)
print(results.shape)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.

In [29]:
import torch.nn as nn
import numpy as np

# SimpleNERModel 클래스 정의: Named Entity Recognition(개체명 인식)을 위한 간단한 모델
# nn.Module을 상속받아 PyTorch 모델로 작동합니다.
class SimpleNERModel(nn.Module):
  # 모델 초기화 메서드
  # num_labels: 개체명 라벨(클래스)의 총 개수
  def __init__(self, num_labels) -> None:
    # 부모 클래스인 nn.Module의 생성자를 호출하여 초기화합니다.
    super(SimpleNERModel, self).__init__()
    # 라벨의 개수를 인스턴스 변수로 저장합니다.
    self.num_labels = num_labels
    # 사전 학습된 BERT 모델을 로드합니다. (MODEL_NAME은 외부에서 정의되어야 합니다.)
    # 이 BERT 모델은 입력 토큰을 임베딩하고 문맥 정보를 학습합니다.
    self.bert = AutoModel.from_pretrained(MODEL_NAME)
    # 드롭아웃 레이어를 정의합니다. 과적합을 방지하기 위해 0.1의 확률로 뉴런을 비활성화합니다.
    self.dropout = nn.Dropout(0.1)
    # 분류를 위한 선형 레이어(Fully Connected Layer)를 정의합니다.
    # BERT 모델의 출력 은닉 상태 크기를 입력으로 받고, 라벨 개수를 출력으로 가집니다.
    self.clf = nn.Linear(self.bert.config.hidden_size,  self.num_labels)

  # 모델의 순전파(forward pass)를 정의하는 메서드
  # input_ids: 입력 토큰 ID 시퀀스
  # attention_mask: 어텐션 마스크 (패딩 토큰을 무시하도록 합니다)
  def forward(self, input_ids, attention_maks):
    # BERT 모델을 사용하여 입력 시퀀스를 인코딩합니다.
    # outputs에는 last_hidden_state, pooler_output 등 다양한 정보가 포함됩니다.
    outputs = self.bert(input_ids, attention_mask=attention_maks)
    # BERT의 마지막 은닉 상태(hidden state)를 추출합니다.
    # 이 상태는 각 입력 토큰에 대한 문맥적 임베딩을 포함합니다.
    sequence_output =  outputs.last_hidden_state
    # 추출된 은닉 상태에 드롭아웃을 적용합니다.
    sequence_output = self.dropout(sequence_output)
    # 드롭아웃이 적용된 은닉 상태를 분류기(선형 레이어)에 통과시켜 로짓(logit)을 계산합니다.
    # 로짓은 각 라벨에 대한 분류 점수를 나타냅니다.
    logits = self.clf(sequence_output)
    # 계산된 로짓을 반환합니다.
    return logits

# 라벨의 고유한 목록을 생성합니다.
# train_labels는 각 샘플의 라벨 시퀀스를 포함하는 리스트의 리스트 형태일 것으로 예상됩니다.
label_list = set([data for i in train_labels for data in i])
# 생성된 라벨 목록을 출력합니다. (디버깅 또는 확인용)
label_list
# SimpleNERModel 인스턴스를 생성합니다.
# 모델의 num_labels는 고유한 라벨의 개수로 설정됩니다.
model = SimpleNERModel(num_labels=len(label_list))

# 모델 학습
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 순전파 테스트
# 순전파 테스트를 위한 코드 블록 시작

# 학습 데이터셋에서 첫 번째 문장과 해당 라벨을 샘플로 가져옵니다.
# train_sentences는 문장들의 리스트이고, 각 문장은 단어들의 리스트로 구성됩니다.
sample_sentence = train_sentences[0]
# train_labels는 라벨 시퀀스들의 리스트이고, 각 라벨 시퀀스는 단어별 라벨들의 리스트로 구성됩니다.
sample_label = train_labels[0]  

# 샘플 문장과 그에 해당하는 정답 라벨을 출력하여 확인합니다.
# ' '.join()을 사용하여 단어 리스트를 공백으로 구분된 하나의 문자열로 만듭니다.
print(f"테스트 문장 {' '.join(sample_sentence)}")
print(f"정답라벨 {' '.join(sample_label)}")

# 토크나이저를 사용하여 샘플 문장을 인코딩합니다.
# return_tensors="pt": PyTorch 텐서 형식으로 반환하도록 지정합니다.
# truncation=True: 최대 길이를 초과하는 시퀀스를 자릅니다.
# max_length=32: 시퀀스의 최대 길이를 32로 설정합니다.
# padding=True: 모든 시퀀스를 max_length에 맞춰 패딩합니다.
# is_split_into_words=True: 입력이 이미 단어 단위로 분리된 리스트임을 토크나이저에게 알려줍니다.
encoding =tokenizer(sample_sentence, return_tensors="pt", truncation=True, 
          max_length=32,padding=True, is_split_into_words=True)
# 인코딩된 input_ids (토큰 ID)를 모델이 있는 장치(CPU 또는 GPU)로 이동시킵니다.
input_ids = encoding["input_ids"].to(device)
# 인코딩된 attention_mask (패딩 토큰을 무시하기 위한 마스크)를 모델이 있는 장치로 이동시킵니다.
attention_masks = encoding["attention_mask"].to(device)

# 모델 평가 모드 설정 및 순전파 수행
# torch.no_grad(): 그래디언트 계산을 비활성화하여 메모리 사용량을 줄이고 계산 속도를 높입니다.
#                  평가 단계에서는 역전파가 필요 없으므로 이 컨텍스트를 사용합니다.
with torch.no_grad():
    # 모델을 평가 모드로 설정합니다. (Dropout, BatchNorm 등의 동작이 평가 모드에 맞게 변경됩니다.)
    model.eval()
    # 모델의 forward 메서드를 호출하여 로짓(logit)을 계산합니다.
    # 로짓은 각 토큰에 대한 각 라벨 클래스의 예측 점수입니다.
    logits = model(input_ids, attention_masks)
    # 로짓에서 가장 높은 값을 가지는 인덱스를 찾아 예측 라벨로 결정합니다.
    # dim=-1: 마지막 차원(클래스 차원)을 기준으로 argmax를 수행합니다.
    predictions = torch.argmax(logits, dim=-1)  

# 토크나이저의 word_ids 메서드를 사용하여 각 토큰이 원본 문장의 어떤 단어에 해당하는지 매핑합니다.
# batch_index=0: 단일 샘플이므로 첫 번째 배치 인덱스를 사용합니다.
word_ids = encoding.word_ids(batch_index=0)
# 예측된 라벨들을 저장할 빈 리스트를 초기화합니다.
pred_labels = []

# 라벨 인덱스를 실제 라벨 문자열로 매핑하기 위한 딕셔너리를 생성합니다.
# label_list는 고유한 라벨 문자열들의 리스트입니다.
id2label = {i: label for i, label in enumerate(label_list)}
# word_ids를 순회하며 각 토큰에 대한 예측 라벨을 원본 단어에 매핑합니다.
for i, word_idx in enumerate(word_ids):
    # word_idx가 None이 아니고 (특수 토큰이 아님) 예측 결과 범위 내에 있을 때만 처리합니다.
    if word_idx is not None and i < len(predictions[0]):
        # 현재 토큰의 예측된 라벨 인덱스를 가져와 id2label 딕셔너리를 통해 실제 라벨 문자열로 변환합니다.
        pred_label = id2label[predictions[0][i].item()]
        # 원본 문장의 단어 인덱스가 유효한 범위 내에 있을 때만 출력합니다.
        if word_idx < len(sample_sentence):
            # 원본 단어, 모델의 예측 라벨, 그리고 실제 정답 라벨을 함께 출력합니다.
            # f-string 포매팅을 사용하여 출력 형식을 맞춥니다.
            print(f"{sample_sentence[word_idx]:10} -> {pred_label:8} 정답 : {sample_label[word_idx]}")

           
# # 데이터 로더 정의
# class NERDataset(Dataset):
    

테스트 문장 김철수는 서울에 산다
정답라벨 B-PER B-LOC O
김철수는       -> O        정답 : B-PER
김철수는       -> B-PER    정답 : B-PER
김철수는       -> O        정답 : B-PER
김철수는       -> O        정답 : B-PER
서울에        -> O        정답 : B-LOC
서울에        -> O        정답 : B-LOC
서울에        -> B-PER    정답 : B-LOC
서울에        -> B-PER    정답 : B-LOC
서울에        -> O        정답 : B-LOC
서울에        -> O        정답 : B-LOC
산다         -> O        정답 : O
산다         -> O        정답 : O
